# Fix HPC Permissions Issues

This notebook helps fix "Operation not permitted" errors on HPC systems.

## 1. Set Custom Temp Directory

The issue is likely that Stan can't write to `/tmp`. Let's use your home directory instead.

In [ ]:
import os
import tempfile

# Create temp directory in your home folder
home_tmp = os.path.expanduser('~/tmp')
os.makedirs(home_tmp, exist_ok=True)

# Set as temp directory
os.environ['TMPDIR'] = home_tmp
os.environ['TEMP'] = home_tmp
os.environ['TMP'] = home_tmp

# Also set Python's tempfile to use it
tempfile.tempdir = home_tmp

print(f"Temp directory set to: {home_tmp}")
print(f"Current tempfile directory: {tempfile.gettempdir()}")

## 2. Check CmdStan Installation

In [ ]:
try:
    import cmdstanpy
    print(f"CmdStanPy version: {cmdstanpy.__version__}")
    
    # Check CmdStan path
    try:
        cmdstan_path = cmdstanpy.cmdstan_path()
        print(f"CmdStan path: {cmdstan_path}")
        
        # Check if it's writable
        test_file = os.path.join(cmdstan_path, '.test_write')
        try:
            with open(test_file, 'w') as f:
                f.write('test')
            os.remove(test_file)
            print("✓ CmdStan directory is writable")
        except Exception as e:
            print(f"✗ CmdStan directory not writable: {e}")
            print("  Consider reinstalling CmdStan in your home directory")
            
    except Exception as e:
        print(f"CmdStan not installed: {e}")
        print("\nInstalling CmdStan in your home directory...")
        cmdstanpy.install_cmdstan(dir=os.path.expanduser('~/.cmdstan'))
        
except ImportError:
    print("CmdStanPy not installed. Installing...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'cmdstanpy'])
    print("Please restart the kernel and run again.")

## 3. Test with Simple Model

In [ ]:
import numpy as np
import pandas as pd
from pydmc import WaldStopSignalModel

# Create minimal test data
np.random.seed(42)
data = []
for i in range(50):
    stimulus = np.random.choice([0, 1])
    response = stimulus + 1
    rt = np.random.uniform(0.4, 0.8)
    data.append({
        'subject': 'S01',
        'stimulus': stimulus,
        'response': response,
        'rt': rt,
        'ssd': np.nan
    })

data = pd.DataFrame(data)
print(f"Test data: {len(data)} trials")

In [ ]:
# Try individual model (simpler)
print("Testing individual model...\n")
model = WaldStopSignalModel(use_hierarchical=False)

try:
    fit = model.fit(
        data,
        chains=1,
        iter=100,
        warmup=50,
        cores=1,
        show_progress=True
    )
    print("\n✓ SUCCESS! Model fitted.")
    model.summary()
    
except Exception as e:
    print(f"\n✗ Error: {e}")
    import traceback
    traceback.print_exc()

## 4. Alternative: Use PyStan Instead

If CmdStanPy continues to have issues, try PyStan:

In [ ]:
# Install PyStan (if needed)
# !pip install pystan==2.19.1.1

# Then pydmc will automatically use it
from pydmc import WaldStopSignalModel
model = WaldStopSignalModel(use_hierarchical=False)
print(f"Backend: {model.backend.backend_name}")

## Troubleshooting Tips

### If still getting permission errors:

1. **Check disk quota**: `quota -s`
2. **Check temp space**: `df -h ~/tmp`
3. **Clear old temp files**: `rm -rf ~/tmp/cmdstan-*`
4. **Run on compute node** (not login node):
   ```bash
   srun --pty bash
   jupyter notebook
   ```

### For HPC batch jobs:

Add to your SLURM script:
```bash
#!/bin/bash
#SBATCH --job-name=pydmc
#SBATCH --time=02:00:00
#SBATCH --mem=8G
#SBATCH --cpus-per-task=4

# Set temp directory
export TMPDIR=$HOME/tmp
mkdir -p $TMPDIR

# Run your script
python your_script.py
```